In [1]:
import numpy as np
import torch
import math
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
import datetime

In [2]:
n=400
d=500

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
mu=np.zeros(d)
sigma=np.zeros((d,d))
sigma[0,1]=0.2
sigma[0,2]=0.2
sigma[0,3]=0.2
sigma[0,4]=0.2

sigma[1,0]=0.2
sigma[1,2]=0.2
sigma[1,3]=0.2
sigma[1,4]=0.2
sigma[1,5]=0.2

sigma[2,0]=0.2
sigma[2,1]=0.2
sigma[2,6]=0.2
sigma[2,3]=0.2
sigma[2,4]=0.2
sigma[2,5]=0.2

sigma[3,0]=0.2
sigma[3,1]=0.2
sigma[3,2]=0.2
sigma[3,6]=0.2
sigma[3,4]=0.2
sigma[3,5]=0.2
sigma[3,7]=0.2

sigma[d-1,d-2]=0.2
sigma[d-1,d-3]=0.2
sigma[d-1,d-4]=0.2
sigma[d-1,d-5]=0.2

sigma[d-2,d-1]=0.2
sigma[d-2,d-3]=0.2
sigma[d-2,d-4]=0.2
sigma[d-2,d-6]=0.2
sigma[d-2,d-5]=0.2

sigma[d-3,d-1]=0.2
sigma[d-3,d-2]=0.2
sigma[d-3,d-4]=0.2
sigma[d-3,d-6]=0.2
sigma[d-3,d-7]=0.2
sigma[d-3,d-5]=0.2

sigma[d-4,d-1]=0.2
sigma[d-4,d-2]=0.2
sigma[d-4,d-3]=0.2
sigma[d-4,d-6]=0.2
sigma[d-4,d-7]=0.2
sigma[d-4,d-8]=0.2
sigma[d-4,d-5]=0.2

for i in range (sigma.shape[0]):
     sigma[i,i]=1
for i in range(4,d-4):
    sigma[i,i-2]=0.2
    sigma[i,i-1]=0.2
    sigma[i,i-3]=0.2
    sigma[i,i-4]=0.2
    sigma[i,i+3]=0.2
    sigma[i,i+4]=0.2
    sigma[i,i+1]=0.2
    sigma[i,i+2]=0.2


In [5]:
W=np.linalg.inv(sigma)
print(W[0,0])

1.133950874544954


In [6]:
def SCAD_dev1(t,a,Self_lamda):
    if (t<=Self_lamda ) :
        dev=Self_lamda 
    else: 
        dev=max(a*Self_lamda-t,0)/(a-1)
    return(dev)

In [7]:
def SCAD_dev2 (intheta,X,Y,n,d,lamda):
    dev=torch.zeros(d,1).to(device) 
    for i in range(dev.shape[0]):
        t=abs(intheta[i,:])
        dev[i,:]=SCAD_dev1(t,3,lamda)
        dev[0,0]=0
    return(dev)

In [8]:
def soft(input_theta,X,Y,n,d,sk,dev):
    A2=Y-X.mm(input_theta)
    A1=torch.t(X).mm(A2)
    a=sk/n
    A=input_theta+A1.mul(a)  
    zero1=torch.zeros(d,1).to(device)
    B=torch.abs(A)-dev.mul(sk)
    C=torch.max(B,zero1)
    D=torch.sign(A)
    output=D.mul(C)
    return(output)

In [9]:
def diff(input,X,Y,n,d,sk,dev):
    new=soft(input,X,Y,n,d,sk,dev)
    di=new-input
    epi_new=Y-X.mm(new)
    epi=Y-X.mm(input)
    ##
    Q=torch.t(epi_new).mm(epi_new.div(2*n))
    ##
    A=torch.t(epi).mm(epi.div(2*n))
    B1=-torch.t(X).mm(epi) 
    B=torch.t(di).mm(B1)
    C=torch.t(di).mm(di)/(2*(sk))
    dif=A+B+C-Q
    return(dif)

In [10]:
def choose_s (input,X,Y,n,d,dev):
    k=1
    dd=diff(input,X,Y,n,d,k,dev)
    while dd<0 :
        k=k*(0.1)
        dd=diff(input,X,Y,n,d,k,dev)
    return(k)

In [11]:
def ISTA (input,X,Y,n,d,dev):
    sk=0.075
    output=soft(input,X,Y,n,d,sk,dev)
    while torch.max(torch.abs(input-output))>1e-5:
        sk=choose_s(input,X,Y,n,d,dev)
        input=output
        output=soft(input,X,Y,n,d,sk,dev)
    return(output)



In [12]:
def LLA (X,Y,lamda,n,d):
    intheta=torch.zeros(d,1).to(device)
    dev = SCAD_dev2(intheta,X,Y,n,d,lamda)
    outtheta= ISTA(intheta,X,Y,n,d,dev)
    while torch.max(torch.abs(intheta-outtheta))>1e-5:
        intheta=outtheta
        outtheta=ISTA(intheta,X,Y,n,d,dev)
        dev=SCAD_dev2(intheta,X,Y,n,d,lamda)
    return(outtheta)

In [13]:
def SCAD(X_cuda,Y_cuda,lamda,n,d):
    start = datetime.datetime.now()
    theta_scad = LLA(X_cuda,Y_cuda,lamda,n,d)
    end = datetime.datetime.now()
    print(end-start)
    return(theta_scad)

In [14]:
def cross_validation(X_cuda,Y_cuda,lamda):
    list1=list(range(0, 80))
    list2=list(range(80, 160))
    list3=list(range(160, 240))
    list4=list(range(240, 320))
    list5=list(range(320, 400))
#############################
    indicex1 = torch.LongTensor(list1).to(device)
    indiceX1 = torch.LongTensor(list2+list3+list4+list5).to(device)
    X1=torch.index_select(X_cuda, 0, indiceX1).to(device)
    x1=torch.index_select(X_cuda, 0, indicex1).to(device)
    Y1=torch.index_select(Y_cuda, 0, indiceX1).to(device)
    y1=torch.index_select(Y_cuda, 0, indicex1).to(device)
    t1= SCAD(X1,Y1,lamda,320,500)
    epi1=y1-x1.mm(t1)
    w1=torch.t(epi1).mm(epi1)
###############################    
    indicex2 = torch.LongTensor(list2).to(device)
    indiceX2 = torch.LongTensor(list1+list3+list4+list5).to(device)
    X2=torch.index_select(X_cuda, 0, indiceX2).to(device)
    x2=torch.index_select(X_cuda, 0, indicex2).to(device)
    Y2=torch.index_select(Y_cuda, 0, indiceX2).to(device)
    y2=torch.index_select(Y_cuda, 0, indicex2).to(device)
    t2= SCAD(X2,Y2,lamda,320,500)
    epi2=y2-x2.mm(t2)
    w2=torch.t(epi2).mm(epi2)
############################ 
    indicex3 = torch.LongTensor(list3).to(device)
    indiceX3 = torch.LongTensor(list2+list1+list4+list5).to(device)
    X3=torch.index_select(X_cuda, 0, indiceX3).to(device)
    x3=torch.index_select(X_cuda, 0, indicex3).to(device)
    Y3=torch.index_select(Y_cuda, 0, indiceX3).to(device)
    y3=torch.index_select(Y_cuda, 0, indicex3).to(device)
    t3= SCAD(X3,Y3,lamda,320,500)
    epi3=y3-x3.mm(t3)
    w3=torch.t(epi3).mm(epi3)
#################################
    indicex4 = torch.LongTensor(list4).to(device)
    indiceX4 = torch.LongTensor(list2+list3+list1+list5).to(device)
    X4=torch.index_select(X_cuda, 0, indiceX4).to(device)
    x4=torch.index_select(X_cuda, 0, indicex4).to(device)
    Y4=torch.index_select(Y_cuda, 0, indiceX4).to(device)
    y4=torch.index_select(Y_cuda, 0, indicex4).to(device)
    t4= SCAD(X4,Y4,lamda,320,500)
    epi4=y4-x4.mm(t4)
    w4=torch.t(epi4).mm(epi4)
################################
    indicex5 = torch.LongTensor(list5).to(device)
    indiceX5 = torch.LongTensor(list2+list3+list4+list1).to(device)
    X5=torch.index_select(X_cuda, 0, indiceX5).to(device)
    x5=torch.index_select(X_cuda, 0, indicex5).to(device)
    Y5=torch.index_select(Y_cuda, 0, indiceX5).to(device)
    y5=torch.index_select(Y_cuda, 0, indicex5).to(device)
    t5= SCAD(X5,Y5,lamda,320,500)
    epi5=y5-x5.mm(t5)
    w5=torch.t(epi5).mm(epi5)
##################################
    error=(w1+w2+w3+w4+w5)/400
    print(error)
    return(error)

In [15]:
def sup(beta1):
    b=torch.zeros(500,1).to(device)
    b[beta1!=0]=1
    s=torch.sum(b)
    return(s-1)

In [16]:
sample = np.random.multivariate_normal(mu, W, n)
X_np=sample.T

In [51]:
i=55
X_copy=X_np
Xj_np=np.zeros((1,400))
Xj_np[0,:]=X_copy[i,:]
X_j_np=np.delete(X_copy,[i],axis=0)
X_j_np.shape
#############################
Xj1=Xj_np.astype("float32")
Xj2=torch.from_numpy(Xj1).to(device)
Xj=torch.t(Xj2)
X_j1=X_j_np.astype("float32")
X_j2=torch.from_numpy(X_j1).to(device)
X_j=torch.t(X_j2)
one=torch.ones(400,1).to(device)
X_j_sum=torch.cat((one,X_j),1)
#####################
beta1=SCAD(X_j_sum,Xj,0.11,400,500)
supp=int(sup(beta1))
print(supp)
#########################
XX=X_j_sum.cpu().detach().numpy()
YY=Xj.cpu().detach().numpy()
Y=np.squeeze(YY)
##############################
reg = OrthogonalMatchingPursuit(supp).fit(XX, Y)
p=reg.coef_
###############################
Z=Y-XX.dot(p)
r=Z.T.dot(Z)/(n-supp)
print(r)

0:00:05.035766
31
0.7403759138132379


In [53]:
beta1=SCAD(X_j_sum,Xj,0.11,400,500)

0:00:04.787332


In [54]:
cross_validation(X_j_sum,Xj,0.1)

0:00:10.098740
0:00:07.286064
0:00:05.424503
0:00:04.840108
0:00:07.918429
tensor([[1.2296]], device='cuda:0')


tensor([[1.2296]], device='cuda:0')

In [55]:
cross_validation(X_j_sum,Xj,0.11)

0:00:09.242736
0:00:06.276570
0:00:05.301283
0:00:03.797206
0:00:06.981582
tensor([[1.2172]], device='cuda:0')


tensor([[1.2172]], device='cuda:0')

In [56]:
cross_validation(X_j_sum,Xj,0.12)

0:00:08.294678
0:00:04.490066
0:00:04.915383
0:00:03.501988
0:00:05.095603
tensor([[1.2079]], device='cuda:0')


tensor([[1.2079]], device='cuda:0')

In [57]:
cross_validation(X_j_sum,Xj,0.13)

0:00:07.092271
0:00:04.114154
0:00:04.659233
0:00:03.368386
0:00:04.585614
tensor([[1.2043]], device='cuda:0')


tensor([[1.2043]], device='cuda:0')

In [58]:
cross_validation(X_j_sum,Xj,0.15)

0:00:04.345991
0:00:03.590000
0:00:03.636580
0:00:03.124518
0:00:03.639458
tensor([[1.2048]], device='cuda:0')


tensor([[1.2048]], device='cuda:0')

In [59]:
cross_validation(X_j_sum,Xj,0.14)

0:00:04.491296
0:00:03.891517
0:00:03.967030
0:00:03.331622
0:00:04.535776
tensor([[1.2045]], device='cuda:0')


tensor([[1.2045]], device='cuda:0')

In [17]:
omega=np.zeros((500,500))

In [23]:
for i in range(0,500):
    start = datetime.datetime.now()
    X_copy=X_np
    Xj_np=np.zeros((1,400))
    Xj_np[0,:]=X_copy[i,:]
    X_j_np=np.delete(X_copy,[i],axis=0)
    X_j_np.shape
    #############################
    Xj1=Xj_np.astype("float32")
    Xj2=torch.from_numpy(Xj1).to(device)
    Xj=torch.t(Xj2)
    X_j1=X_j_np.astype("float32")
    X_j2=torch.from_numpy(X_j1).to(device)
    X_j=torch.t(X_j2)
    one=torch.ones(400,1).to(device)
    X_j_sum=torch.cat((one,X_j),1)
    #####################
    beta1=SCAD(X_j_sum,Xj,0.13,400,500)
    supp=int(sup(beta1))
#     print(supp)
    #########################
    XX=X_j_sum.cpu().detach().numpy()
    YY=Xj.cpu().detach().numpy()
    Y=np.squeeze(YY)
    ##############################
    reg = OrthogonalMatchingPursuit(supp).fit(XX, Y)
    p=reg.coef_
    ###############################
    Z=Y-XX.dot(p)
    r=Z.T.dot(Z)/(n-supp)
#     print(r)
    beta=beta1.cpu().detach().numpy()
    w=beta/r
    w[0]=1/r
    read=np.zeros((500,1))
    for k in range(0,i):
        read[k]=w[k+1]
    read[i]=w[0]
    for k in range(i+1,500):
        read[k]=w[k]
    omega[:,i]=read[:,0]
    end = datetime.datetime.now()
    print("loop",i,end-start)

0:00:05.863472
loop 0 0:00:05.886720
0:00:04.317782
loop 1 0:00:04.338641
0:00:05.995821
loop 2 0:00:06.010079
0:00:06.854970
loop 3 0:00:06.889887
0:00:06.018093
loop 4 0:00:06.036713
0:00:10.447552
loop 5 0:00:10.459444
0:00:06.890229
loop 6 0:00:06.904467
0:00:08.612485
loop 7 0:00:08.627773
0:00:05.915162
loop 8 0:00:05.995983
0:00:10.285077
loop 9 0:00:10.297342
0:00:06.838590
loop 10 0:00:06.849095
0:00:06.138563
loop 11 0:00:06.146728
0:00:06.808760
loop 12 0:00:06.816541
0:00:08.202808
loop 13 0:00:08.213431
0:00:06.050170
loop 14 0:00:06.058757
0:00:05.637053
loop 15 0:00:05.645439
0:00:04.534756
loop 16 0:00:04.541788
0:00:06.594559
loop 17 0:00:06.602353
0:00:05.223322
loop 18 0:00:05.230561
0:00:06.088098
loop 19 0:00:06.095370
0:00:06.689429
loop 20 0:00:06.699097
0:00:06.150173
loop 21 0:00:06.160092
0:00:05.794467
loop 22 0:00:05.801238
0:00:07.250468
loop 23 0:00:07.258176
0:00:07.020433
loop 24 0:00:07.028022
0:00:06.358741
loop 25 0:00:06.367367
0:00:07.779949
loop 26

In [24]:
np.save("math287model2new",omega)

In [25]:
rt=np.load("math287model2new.npy")

In [26]:
ee=rt
for i in range(0,500):
    for j in range(i+1,500):
        if (np.abs(ee[i,j])<=np.abs(ee[j,i])):
            ee[j,i]=ee[i,j]
        else:
            ee[i,j]=ee[j,i]

In [28]:
print("F:",np.linalg.norm(dd,ord = 'fro'),"Operator；",np.linalg.norm(dd,ord = 2),"1,inf:",np.linalg.norm(dd,ord = np.inf))

F: 16.412533466447986 Operator； 1.8066756977504579 1,inf: 3.153419783827849


In [29]:
def TP(KK):
    summ=0
    for i in range(4,496):
        for j in range(i-4,i+5):
            if(KK[i,j]!=0):
                summ=summ+1
                KK[i,j]=0
    u=(summ-10)/(9*500-20)
    return(u)
    

In [30]:
TP(ee)

0.6720982142857143

In [31]:
def TN(KK):
    for i in range(4,496):
        for j in range(i-4,i+5):
                KK[i,j]=0
    summ=0             
    for i in range(0,500):
        for j in range(0,500):
            if(KK[i,j]!=0):
                summ=summ+1
    u=(summ-10)/(500*500-4480)
    return(1-u)

In [32]:
TN(ee)

0.9810891169762137